In [1]:
import pickle

import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open3d as o3d
from sklearn.decomposition import PCA
from scipy.spatial import distance

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation

data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_number = "000007"
curr_frame_number = "000007"
new_frame_number = "000008"
patch_h= 18
image_height = 540
image_width = 960
patch_w = 32
patch_height = 30
patch_width = 30

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/local/home/ekoller/BT


In [2]:
def visualize(mask, new_ids, title):
    plt.figure(figsize=(12, 6))

    # Display the mask
    plt.subplot(1, 2, 1)
    plt.imshow(mask, cmap='gray')
    plt.title('Mask')
    plt.axis('off')

    # Display the new_ids array
    plt.subplot(1, 2, 2)
    plt.imshow(new_ids, cmap='gray')
    plt.title(title)
    plt.axis('off')

    plt.show()

#visualitation of the following: with the new ids fill in an image with the new ids
#compute the resulting image when comparing stuff

def generate_pairs_pixel_level( dino_data,id_matches):
    
    new_ids = np.zeros((image_height, image_width))


    #go over every mask and fill in the id into the new_ids which it got mapped to
    for seg_region in dino_data:
        mask_id = seg_region["object_id"]
        print("mask id ", mask_id)
        #get to what the region mapped in the embeddings
        matched_id = id_matches[mask_id]
        print("matched id ", matched_id)
        mask = seg_region["mask"]
        boolean_mask = mask == 225
        new_ids[boolean_mask] = matched_id[0] #[0] is the id the second one is the error
        #visualize(boolean_mask, new_ids, f'Updated new_ids with mask id {mask_id}')

    #returns the new ids on a pixel wise level
    return new_ids


In [3]:
#this codesegment takes an image on a pixel wise level and quantizes it such that every patch has only the id of the most often occuring id
def quantize_to_patch_level(pixelwise_img):
    #get the shape of the pixelwise img
    input_h, input_w = pixelwise_img.shape
    patch_width = int(input_w/patch_w)
    patch_height= int(input_h/patch_h)

    patchwise_w = patch_w #number of patches
    patchwise_h = patch_h

    patchwise_id = np.zeros((patchwise_h,patchwise_w))

    for i in range(patchwise_h):
            for j in range(patchwise_w):
                # Define the coordinates of the current patch
                h_start = i * patch_height
                w_start = j * patch_width
                h_end = h_start + patch_height
                w_end = w_start + patch_width
                
                # Get the current patch from the input matrix
                patch = pixelwise_img[h_start:h_end, w_start:w_end]
                
                # get the most reoccuring id of the patch
                flattened_patch = patch.flatten()
                # Find the most common value in the patch
                value_counts = Counter(flattened_patch)
                most_common_id = value_counts.most_common(1)[0][0]
                
                # Assign the most common ID to the new matrix
                patchwise_id[i, j] = most_common_id


    return patchwise_id


In [4]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color[::-1])
    
    return object_colors

colors = get_id_colours(data_dir,curr_scan_id)
print("cur scan id colors", colors.keys())
colors = get_id_colours(data_dir,new_scan_id)
print("new scan id colors", colors.keys())


cur scan id colors dict_keys([4, 18, 2, 0, 3, 1, 100, 16, 8, 9, 24, 25, 29, 6, 26, 10, 11, 23, 27, 22, 5, 20, 12, 15, 13, 14, 103, 21])
new scan id colors dict_keys([13, 20, 1, 4, 100, 6, 14, 15, 18, 29, 5, 8, 16, 105, 0])


In [5]:
#this function creates a color image of the size 960x540 from the patches
def create_color_img_from_obj_id(data_dir,scan_id,obj_id_mat):
    #access the mesh file to get the colour of the ids
    colour_dict = get_id_colours(data_dir, scan_id)
    #initialize the new matrix 
    og_height, og_width = obj_id_mat.shape
    new_height = og_height * patch_height
    new_width = og_width * patch_width

    colour_mat = np.zeros((new_height,new_width,3))

    #go over each element and assign the colour of the dictionary
    for h in range(og_height):
        for w in range(og_width):
            if obj_id_mat[h][w] in colour_dict.keys():
                colour = colour_dict[obj_id_mat[h][w]]
                colour_mat[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width] = colour


    return colour_mat


    

In [6]:
#given a gt patchwise matrix and a newly calculated matrix compute the accuracy
#aaparently this is not the same as accuracy
#given a gt patchwise matrix and a newly calculated matrix compute the accuracy
def get_accuracy(gt_patches, computed_patches):
    #make sure we dont do something dumm lol
    assert gt_patches.shape == computed_patches.shape, "Matrices must have the same shape"

    # Flatten matrices to iterate over each element
    flat_gt = gt_patches.flatten()
    flat_comp = computed_patches.flatten()

    percentage = np.zeros_like(flat_gt)
    
    total_instances = len(flat_gt)
    correct_instances = 0

    #compute based on the other way
    for idx in range(len(percentage)):
        if flat_gt[idx] == flat_comp[idx]:
            percentage[idx] = 1
    
    # Count correct instances where IDs match
    for id_gt, id_comp in zip(flat_gt, flat_comp):
        if id_gt == id_comp:
            correct_instances += 1
            
    
    # Compute accuracy
    accuracy = correct_instances / total_instances
    
    return accuracy, np.mean(percentage)

In [ ]:

def avg_pooling(all_features_curr_scene):
    #get every fearure vector and average the patches to one patch
    all_features = []
    all_colors = []
    for frame in all_features_curr_scene:
        for obj_id in all_features_curr_scene[frame]:
            if obj_id != 0:
                matrix = all_features_curr_scene[frame][obj_id]
                #averaging
                aggregated_feature = np.mean(matrix, axis=0)  
                all_features.append(aggregated_feature)

                all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

    return all_features, all_colors

def max_pooling(all_features_curr_scene):
    #perform max pooling 
    all_features = []
    all_colors = []
    for frame in all_features_curr_scene:
        for obj_id in all_features_curr_scene[frame]:
            if obj_id != 0:
                matrix = all_features_curr_scene[frame][obj_id]

                #max pooling
                aggregated_feature = np.max(matrix, axis=0)  # Max pooling across patches
                all_features.append(aggregated_feature)

                # append color corresponding to the object id
                all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

    return all_features, all_colors

The following function does this: given an input image of the new_scan id, get the features of all the objects & the objects of the curr_scene. Then compute the closest vector and assign each segment a object id of the current scan. No threshold yet

In [25]:

"""
for the current scene: access all the precomputed dinov2 features and do pca of the space
"""
#get all the features of the current scene
all_info_path = osp.join(data_dir,"files/Features2D/projection/DinoV2/patch_32_18", curr_scan_id + ".pkl")
with open(all_info_path, 'rb') as file:
    all_features_curr_scene = pickle.load(file)

#using avg pooling
all_features, all_colors = avg_pooling(all_features_curr_scene)
all_features = np.array(all_features)
all_colors = np.array(all_colors)

#perform pca to 3D reduction
pca = PCA(n_components=3)
reduced_points = pca.fit_transform(all_features)


"""
access the features of the new scene and the info
"""

#get the Dinov2 features for the input image ( the segmented one)
input_featues_path = osp.join(data_dir,"files/Features2D/dino_segmentation/DinoV2/patch_32_18", new_scan_id + ".pkl")
with open(input_featues_path, 'rb') as file:
    input_features = pickle.load(file)

#get the data information on the segmentation of the input image
input_info_path = osp.join(data_dir,"files/Segmentation/DinoV2/objects", new_scan_id + ".pkl")
with open(input_info_path, 'rb') as file:
    input_info = pickle.load(file)


id_matches= []

#iterate over everything and give back the closest match for every box in sam
def of_all_find_closes_pairs(sg_dict, img_dict):
    closest_pairs = {}
    print("sg_data object ids of image", sg_dict.keys())
    print("img_data object ids of image", img_dict.keys())

    # print("input data sg dict", sg_dict)
    # print("input data img dict", img_dict)
    #go over the image dict since we want to get the closest match of all the ids
    for img_id, img_vec in img_dict.items():
        min_distance = -1
        closest_id = None
    
        
        # go over every vector in the scenegraph
        for obj_id, obj_vec in sg_dict.items():
        
            #print("obj id vec shape", obj_vec.shape)
            #print("img id vec shape", img_vec.shape)
            cosine_similarity_all_patches = [cosine_similarity(obj_vec[i], img_vec[i]) for i in range(img_vec.shape[0])]
            average_cosine_similarity = np.mean(cosine_similarity_all_patches)
          
            
            #update
            if average_cosine_similarity> min_distance:
                min_distance = average_cosine_similarity
                closest_id= obj_id
        #check if it is close enoug using a threshold
        if min_distance > 0:
            closest_pairs[img_id] = (closest_id, min_distance)
        else:
            closest_pairs[img_id] = (0, -1)

        #closest_pairs[img_id] = (closest_id, min_distance)

    return closest_pairs


"""
compute the best match for each input on a patchwise level
"""
#closest pairs  fucking important lol !!!!!!!
id_matches = of_all_find_closes_pairs(curr_features[frame_number], input_features[new_frame_number])
print("id mateches", id_matches)

#assign each pixel to the new value, curr_scan_id , frame number is only for the size
new_img_pixelwise = generate_pairs_pixel_level(input_info[frame_number], id_matches)

#quantize to patches
new_img_patchwise = quantize_to_patch_level(new_img_pixelwise)
#aggregate to patches and colour it, we want the colours which are used in curr_scan id
new_img_colour = create_color_img_from_obj_id(data_dir,curr_scan_id, new_img_patchwise)
print("unique numbers in new img patchwise ", np.unique(new_img_patchwise))

#get the gt patches for the segmented scene! so for the dinov2 segmentation: but the colour will be based on the current scene
gt_input_patchwise_path =  osp.join(data_dir,"files/patch_anno/patch_anno_32_18", new_scan_id + '.pkl')
with open(gt_input_patchwise_path, 'rb') as file:
    gt_input_patchwise = pickle.load(file)
print("unique numbers in gt_input_colour ", np.unique(gt_input_patchwise[frame_number]))
gt_input_colour = create_color_img_from_obj_id(data_dir,curr_scan_id,gt_input_patchwise[frame_number])




accuracy = get_accuracy(gt_input_patchwise[frame_number],new_img_patchwise)
print("the accuracy for the current features of the sam boundingboxes vs the gt boundingboxes is", accuracy)


"""
display the newly computed images next to each other
"""

#display the gt_image and the new patchwise image next to each other
title1 = "gt_patces"
title2 = "dino_seg_patches"

# Create a blank canvas to combine images horizontally
height = max(gt_input_colour.shape[0], new_img_colour.shape[0])  # Max height of both images
width = gt_input_colour.shape[1] + new_img_colour.shape[1] + 20  # Total width of both images with a small gap
combined_image = np.zeros((height, width, 3), dtype=np.uint8)

# Place images with titles on the blank canvas
combined_image[:gt_input_colour.shape[0], :gt_input_colour.shape[1]] = gt_input_colour
combined_image[:new_img_colour.shape[0], gt_input_colour.shape[1] + 20:] = new_img_colour

# Add titles to the images
cv2.putText(combined_image, title1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(combined_image, title2, (gt_input_colour.shape[1] + 30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Display the combined image
cv2.imshow('Two Images Side by Side', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()




sg_data object ids of image dict_keys([0, 1, 4, 13, 14, 15, 18])
img_data object ids of image dict_keys([1, 2, 3, 4, 5, 6, 7, 8])
abs 1 [0.00507749 0.01847113 0.03231867 ... 0.02026465 0.00596228 0.02972083]
abs 1 [0.00507749 0.01847113 0.03231867 ... 0.02026465 0.00596228 0.02972083]
norm 2 1.0000004
abs 1 [0.03627722 0.01382175 0.04336117 ... 0.04160173 0.02364097 0.04230888]
abs 1 [0.03627722 0.01382175 0.04336117 ... 0.04160173 0.02364097 0.04230888]
norm 2 0.99999994
abs 1 [0.01718338 0.03161878 0.00528147 ... 0.00331493 0.00193568 0.00770423]
abs 1 [0.01718338 0.03161878 0.00528147 ... 0.00331493 0.00193568 0.00770423]
norm 2 1.0000004
abs 1 [0.03511454 0.01101242 0.0386068  ... 0.03178781 0.02495868 0.03201694]
abs 1 [0.03511454 0.01101242 0.0386068  ... 0.03178781 0.02495868 0.03201694]
norm 2 0.9999997
abs 1 [0.02212662 0.02165816 0.00948296 ... 0.0014459  0.01032536 0.02428957]
abs 1 [0.02212662 0.02165816 0.00948296 ... 0.0014459  0.01032536 0.02428957]
norm 2 0.99999994
abs